# Speculative Decoding

SGLang now provides an EAGLE-based (EAGLE-2/EAGLE-3) speculative decoding option. Our implementation aims to maximize speed and efficiency and is considered to be among the fastest in open-source LLM engines.

### Performance Highlights

Please see below for the huge improvements on throughput for LLaMA-Instruct 3.1 8B tested on MT bench that can be achieved via EAGLE3 decoding.
For further details please see the [EAGLE3 paper](https://arxiv.org/pdf/2503.01840).

| Method | Throughput (tokens/s) |
|--------|----------------|
| SGLang (w/o speculative, 1x H100) | 158.34 tokens/s |
| SGLang + EAGLE-2 (1x H100) | 244.10 tokens/s |
| SGLang + EAGLE-3 (1x H100) | 373.25 tokens/s |

## EAGLE Decoding

To enable EAGLE speculative decoding the following parameters are relevant:
* `speculative_draft_model_path`: Specifies draft model. This parameter is required.
* `speculative_num_steps`: Depth of autoregressive drafting. Increases speculation range but risks rejection cascades. Default is 5.
* `speculative_eagle_topk`: Branching factor per step. Improves candidate diversity, will lead to higher acceptance rate, but more lead to higher memory/compute consumption. Default is 4.
* `speculative_num_draft_tokens`: Maximum parallel verification capacity. Allows deeper tree evaluation but will lead to higher GPU memory usage. Default is 8.

These parameters are the same for EAGLE-2 and EAGLE-3.

You can find the best combinations of these parameters with [bench_speculative.py](https://github.com/sgl-project/sglang/blob/main/scripts/playground/bench_speculative.py).

In the documentation below, we set `--cuda-graph-max-bs` to be a small value for faster engine startup. For your own workloads, please tune the above parameters together with `--cuda-graph-max-bs`, `--max-running-requests`, `--mem-fraction-static` for the best performance. 

### EAGLE-2 decoding

You can enable EAGLE-2 decoding by setting `--speculative-algorithm EAGLE` and choosing an appropriate model.

In [1]:
from sglang.test.doc_patch import launch_server_cmd
from sglang.utils import wait_for_server, print_highlight, terminate_process

import openai

In [2]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 3 \
    --speculative-eagle-topk 4 --speculative-num-draft-tokens 16 --cuda-graph-max-bs 8
"""
)

wait_for_server(f"http://localhost:{port}")

W0822 06:03:54.376000 1532160 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0822 06:03:54.376000 1532160 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-22 06:03:55] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=32921, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.849, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=81707

[2025-08-22 06:03:55] Using default HuggingFace chat template with detected content format: string


W0822 06:04:03.076000 1533129 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0822 06:04:03.076000 1533129 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0822 06:04:03.076000 1533130 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0822 06:04:03.076000 1533130 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-22 06:04:03] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-08-22 06:04:03] Init torch distributed begin.


...

Capturing batches (bs=1 avail_mem=53.66 GB): 100%|██████████| 4/4 [00:09<00:00,  2.38s/it]
[2025-08-22 06:04:22] Capture draft cuda graph end. Time elapsed: 10.13 s. mem usage=-19.43 GB. avail mem=52.05 GB.
[2025-08-22 06:04:22] Capture draft extend cuda graph begin. This can take up to several minutes. avail mem=52.05 GB


Capturing batches (bs=2 avail_mem=50.63 GB):   0%|          | 0/4 [00:00<?, ?it/s]

Capturing batches (bs=1 avail_mem=50.61 GB): 100%|██████████| 4/4 [00:00<00:00, 13.97it/s]
[2025-08-22 06:04:23] Capture draft extend cuda graph end. Time elapsed: 1.04 s. mem usage=1.61 GB. avail mem=50.44 GB.
[2025-08-22 06:04:23] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=4096, available_gpu_mem=50.44 GB
[2025-08-22 06:04:23] INFO:     Started server process [1532160]
[2025-08-22 06:04:23] INFO:     Waiting for application startup.
[2025-08-22 06:04:24] INFO:     Application startup complete.
[2025-08-22 06:04:24] INFO:     Uvicorn running on http://127.0.0.1:32921 (Press CTRL+C to quit)


[2025-08-22 06:04:24] INFO:     127.0.0.1:54398 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-22 06:04:25] INFO:     127.0.0.1:33158 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-22 06:04:25] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-22 06:04:25] INFO:     127.0.0.1:33168 - "POST /generate HTTP/1.1" 200 OK
[2025-08-22 06:04:25] The server is fired up and ready to roll!


In [3]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-22 06:04:29] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-22 06:04:29] INFO:     127.0.0.1:33176 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [4]:
terminate_process(server_process)

### EAGLE-2 Decoding with `torch.compile`

You can also enable `torch.compile` for further optimizations and optionally set `--torch-compile-max-bs`:


In [5]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-2-7b-chat-hf  --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-llama2-chat-7B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --mem-fraction 0.6 \
            --enable-torch-compile --torch-compile-max-bs 2
"""
)

wait_for_server(f"http://localhost:{port}")

W0822 06:04:36.711000 1536454 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0822 06:04:36.711000 1536454 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-22 06:04:38] server_args=ServerArgs(model_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_path='meta-llama/Llama-2-7b-chat-hf', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=34208, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.6, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=9701311

[2025-08-22 06:04:38] Using default HuggingFace chat template with detected content format: string


W0822 06:04:45.603000 1537234 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0822 06:04:45.603000 1537234 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0822 06:04:45.713000 1537235 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0822 06:04:45.713000 1537235 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-22 06:04:46] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-08-22 06:04:46] Init torch distributed begin.


...

Capturing batches (bs=1 avail_mem=36.13 GB): 100%|██████████| 4/4 [00:00<00:00, 45.64it/s]
[2025-08-22 06:05:50] Capture draft extend cuda graph end. Time elapsed: 0.83 s. mem usage=0.10 GB. avail mem=36.13 GB.
[2025-08-22 06:05:50] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=4096, available_gpu_mem=36.13 GB


[2025-08-22 06:05:50] INFO:     Started server process [1536454]
[2025-08-22 06:05:50] INFO:     Waiting for application startup.
[2025-08-22 06:05:50] INFO:     Application startup complete.
[2025-08-22 06:05:50] INFO:     Uvicorn running on http://127.0.0.1:34208 (Press CTRL+C to quit)


[2025-08-22 06:05:50] INFO:     127.0.0.1:43882 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-22 06:05:51] INFO:     127.0.0.1:43890 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-22 06:05:51] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-22 06:05:53] INFO:     127.0.0.1:43896 - "POST /generate HTTP/1.1" 200 OK
[2025-08-22 06:05:53] The server is fired up and ready to roll!


In [6]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Llama-2-7b-chat-hf",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-22 06:05:55] Prefill batch. #new-seq: 1, #new-token: 16, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-22 06:05:56] INFO:     127.0.0.1:38102 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [7]:
terminate_process(server_process)

### EAGLE-2 Decoding via Frequency-Ranked Speculative Sampling

By employing a truncated high-frequency token vocabulary in the draft model, Eagle speculative decoding reduces `lm_head` computational overhead while accelerating the pipeline without quality degradation. For more details, checkout [the paper](https://arxiv.org/pdf/arXiv:2502.14856).

In our implementation, set `--speculative-token-map` to enable the optimization. You can get the high-frequency token in FR-Spec from [this model](https://huggingface.co/thunlp/LLaMA3-Instruct-8B-FR-Spec). Or you can obtain high-frequency token by directly downloading these token from [this repo](https://github.com/thunlp/FR-Spec/tree/main?tab=readme-ov-file#prepare-fr-spec-vocabulary-subset).

Thanks for the contribution from [Weilin Zhao](https://github.com/Achazwl) and [Zhousx](https://github.com/Zhou-sx). 

In [8]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Meta-Llama-3-8B-Instruct --speculative-algorithm EAGLE \
    --speculative-draft-model-path lmsys/sglang-EAGLE-LLaMA3-Instruct-8B --speculative-num-steps 5 \
    --speculative-eagle-topk 8 --speculative-num-draft-tokens 64 --speculative-token-map thunlp/LLaMA3-Instruct-8B-FR-Spec/freq_32768.pt \
    --mem-fraction 0.7 --cuda-graph-max-bs 2 --dtype float16 
"""
)

wait_for_server(f"http://localhost:{port}")

W0822 06:06:03.747000 1546076 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0822 06:06:03.747000 1546076 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-22 06:06:04] server_args=ServerArgs(model_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_path='meta-llama/Meta-Llama-3-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=39942, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='float16', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.7, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, rand

[2025-08-22 06:06:05] Using default HuggingFace chat template with detected content format: string


W0822 06:06:12.499000 1546663 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0822 06:06:12.499000 1546663 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0822 06:06:12.625000 1546662 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0822 06:06:12.625000 1546662 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-22 06:06:13] Casting torch.bfloat16 to torch.float16.


...

Capturing batches (bs=1 avail_mem=3.74 GB): 100%|██████████| 4/4 [00:06<00:00,  1.67s/it]
[2025-08-22 06:06:42] Capture draft cuda graph end. Time elapsed: 7.89 s. mem usage=1.05 GB. avail mem=3.70 GB.
[2025-08-22 06:06:42] Capture draft extend cuda graph begin. This can take up to several minutes. avail mem=3.70 GB


Capturing batches (bs=1 avail_mem=3.60 GB): 100%|██████████| 4/4 [00:00<00:00, 91.32it/s]
[2025-08-22 06:06:43] Capture draft extend cuda graph end. Time elapsed: 0.68 s. mem usage=0.10 GB. avail mem=3.60 GB.
[2025-08-22 06:06:43] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=8192, available_gpu_mem=3.60 GB


[2025-08-22 06:06:44] INFO:     Started server process [1546076]
[2025-08-22 06:06:44] INFO:     Waiting for application startup.
[2025-08-22 06:06:44] INFO:     Application startup complete.
[2025-08-22 06:06:44] INFO:     Uvicorn running on http://127.0.0.1:39942 (Press CTRL+C to quit)


[2025-08-22 06:06:44] INFO:     127.0.0.1:39914 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-22 06:06:45] INFO:     127.0.0.1:46376 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-22 06:06:45] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-22 06:06:45] INFO:     127.0.0.1:46392 - "POST /generate HTTP/1.1" 200 OK
[2025-08-22 06:06:45] The server is fired up and ready to roll!


In [9]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-22 06:06:49] Prefill batch. #new-seq: 1, #new-token: 17, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-22 06:06:50] INFO:     127.0.0.1:46404 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [10]:
terminate_process(server_process)

### EAGLE-3 Decoding

You can enable EAGLE-3 decoding by setting `--speculative-algorithm EAGLE3` and choosing an appropriate model.

In [11]:
server_process, port = launch_server_cmd(
    """
python3 -m sglang.launch_server --model meta-llama/Llama-3.1-8B-Instruct  --speculative-algorithm EAGLE3 \
    --speculative-draft-model-path jamesliu1/sglang-EAGLE3-Llama-3.1-Instruct-8B --speculative-num-steps 5 \
        --speculative-eagle-topk 8 --speculative-num-draft-tokens 32 --mem-fraction 0.6 \
        --cuda-graph-max-bs 2 --dtype float16
"""
)

wait_for_server(f"http://localhost:{port}")

W0822 06:06:57.237000 1549742 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0822 06:06:57.237000 1549742 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-22 06:06:57] server_args=ServerArgs(model_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_path='meta-llama/Llama-3.1-8B-Instruct', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=False, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='127.0.0.1', port=30203, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='float16', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.6, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_see

[2025-08-22 06:06:58] Using default HuggingFace chat template with detected content format: string


W0822 06:07:05.562000 1550106 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0822 06:07:05.562000 1550106 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0822 06:07:05.562000 1550107 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0822 06:07:05.562000 1550107 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-22 06:07:06] Casting torch.bfloat16 to torch.float16.


...

Capturing batches (bs=1 avail_mem=38.78 GB): 100%|██████████| 4/4 [00:04<00:00,  1.23s/it]
[2025-08-22 06:07:31] Capture draft cuda graph end. Time elapsed: 5.41 s. mem usage=0.33 GB. avail mem=38.74 GB.
[2025-08-22 06:07:31] Capture draft extend cuda graph begin. This can take up to several minutes. avail mem=38.74 GB


Capturing batches (bs=1 avail_mem=38.63 GB): 100%|██████████| 4/4 [00:00<00:00, 49.11it/s]
[2025-08-22 06:07:32] Capture draft extend cuda graph end. Time elapsed: 0.65 s. mem usage=0.11 GB. avail mem=38.63 GB.
[2025-08-22 06:07:32] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=131072, available_gpu_mem=38.63 GB


[2025-08-22 06:07:33] INFO:     Started server process [1549742]
[2025-08-22 06:07:33] INFO:     Waiting for application startup.
[2025-08-22 06:07:33] INFO:     Application startup complete.
[2025-08-22 06:07:33] INFO:     Uvicorn running on http://127.0.0.1:30203 (Press CTRL+C to quit)
[2025-08-22 06:07:33] INFO:     127.0.0.1:46312 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-22 06:07:34] INFO:     127.0.0.1:46326 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-22 06:07:34] Prefill batch. #new-seq: 1, #new-token: 7, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-22 06:07:34] INFO:     127.0.0.1:46328 - "POST /generate HTTP/1.1" 200 OK
[2025-08-22 06:07:34] The server is fired up and ready to roll!


In [12]:
client = openai.Client(base_url=f"http://127.0.0.1:{port}/v1", api_key="None")

response = client.chat.completions.create(
    model="meta-llama/Meta-Llama-3.1-8B-Instruct",
    messages=[
        {"role": "user", "content": "List 3 countries and their capitals."},
    ],
    temperature=0,
    max_tokens=64,
)

print_highlight(f"Response: {response}")

[2025-08-22 06:07:38] Prefill batch. #new-seq: 1, #new-token: 42, #cached-token: 1, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-22 06:07:38] INFO:     127.0.0.1:33804 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [13]:
terminate_process(server_process)

## Multi Token Prediction

We support [MTP(Multi-Token Prediction)](https://arxiv.org/pdf/2404.19737) in SGLang by using speculative decoding. We use Xiaomi/MiMo-7B-RL model as example here (deepseek mtp usage refer to [deepseek doc](../references/deepseek.md#multi-token-prediction))

In [14]:
server_process, port = launch_server_cmd(
    """
    python3 -m sglang.launch_server --model-path XiaomiMiMo/MiMo-7B-RL --host 0.0.0.0 --trust-remote-code \
    --speculative-algorithm EAGLE --speculative-num-steps 1 --speculative-eagle-topk 1 --speculative-num-draft-tokens 2 \
    --mem-fraction 0.5
"""
)

wait_for_server(f"http://localhost:{port}")

W0822 06:07:46.330000 1552215 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0822 06:07:46.330000 1552215 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-22 06:07:47] server_args=ServerArgs(model_path='XiaomiMiMo/MiMo-7B-RL', tokenizer_path='XiaomiMiMo/MiMo-7B-RL', tokenizer_mode='auto', skip_tokenizer_init=False, load_format='auto', model_loader_extra_config='{}', trust_remote_code=True, context_length=None, is_embedding=False, enable_multimodal=None, revision=None, model_impl='auto', host='0.0.0.0', port=37404, skip_server_warmup=False, warmups=None, nccl_port=None, dtype='auto', quantization=None, quantization_param_path=None, kv_cache_dtype='auto', mem_fraction_static=0.5, max_running_requests=128, max_queued_requests=9223372036854775807, max_total_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, schedule_policy='fcfs', schedule_conservativeness=1.0, cpu_offload_gb=0, page_size=1, hybrid_kvcache_ratio=None, swa_full_tokens_ratio=0.8, disable_hybrid_swa_memory=False, device='cuda', tp_size=1, pp_size=1, max_micro_batch_size=None, stream_interval=1, stream_output=False, random_seed=843892948, constrained_jso

[2025-08-22 06:07:47] Using default HuggingFace chat template with detected content format: string


W0822 06:07:55.375000 1553050 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0822 06:07:55.375000 1553050 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.
W0822 06:07:55.523000 1553051 torch/utils/cpp_extension.py:2425] TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
W0822 06:07:55.523000 1553051 torch/utils/cpp_extension.py:2425] If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'] to specific architectures.


[2025-08-22 06:07:56] Attention backend not explicitly specified. Use flashinfer backend by default.
[2025-08-22 06:07:56] Init torch distributed begin.


...

Capturing batches (bs=1 avail_mem=42.28 GB): 100%|██████████| 4/4 [00:01<00:00,  2.01it/s]
[2025-08-22 06:08:06] Capture draft cuda graph end. Time elapsed: 2.54 s. mem usage=-4.99 GB. avail mem=42.28 GB.
[2025-08-22 06:08:06] Capture draft extend cuda graph begin. This can take up to several minutes. avail mem=42.28 GB


Capturing batches (bs=1 avail_mem=42.18 GB): 100%|██████████| 4/4 [00:00<00:00, 36.77it/s]
[2025-08-22 06:08:07] Capture draft extend cuda graph end. Time elapsed: 0.78 s. mem usage=0.09 GB. avail mem=42.18 GB.
[2025-08-22 06:08:07] max_total_num_tokens=20480, chunked_prefill_size=8192, max_prefill_tokens=16384, max_running_requests=128, context_len=32768, available_gpu_mem=42.18 GB


[2025-08-22 06:08:07] INFO:     Started server process [1552215]
[2025-08-22 06:08:07] INFO:     Waiting for application startup.
[2025-08-22 06:08:07] INFO:     Application startup complete.
[2025-08-22 06:08:07] INFO:     Uvicorn running on http://0.0.0.0:37404 (Press CTRL+C to quit)


[2025-08-22 06:08:08] INFO:     127.0.0.1:41486 - "GET /v1/models HTTP/1.1" 200 OK


[2025-08-22 06:08:08] INFO:     127.0.0.1:41494 - "GET /get_model_info HTTP/1.1" 200 OK
[2025-08-22 06:08:08] Prefill batch. #new-seq: 1, #new-token: 6, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-22 06:08:08] INFO:     127.0.0.1:41498 - "POST /generate HTTP/1.1" 200 OK
[2025-08-22 06:08:08] The server is fired up and ready to roll!


In [15]:
import requests

url = f"http://localhost:{port}/v1/chat/completions"

data = {
    "model": "XiaomiMiMo/MiMo-7B-RL",
    "messages": [{"role": "user", "content": "What is the capital of France?"}],
}

response = requests.post(url, json=data)
print_highlight(response.json())

[2025-08-22 06:08:13] Prefill batch. #new-seq: 1, #new-token: 26, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-22 06:08:13] Decode batch. #running-req: 1, #token: 91, token usage: 0.00, accept len: 1.76, cuda graph: True, gen throughput (token/s): 11.63, #queue-req: 0, 


[2025-08-22 06:08:13] Decode batch. #running-req: 1, #token: 159, token usage: 0.01, accept len: 1.70, cuda graph: True, gen throughput (token/s): 175.30, #queue-req: 0, 


[2025-08-22 06:08:14] Decode batch. #running-req: 1, #token: 224, token usage: 0.01, accept len: 1.62, cuda graph: True, gen throughput (token/s): 166.22, #queue-req: 0, 


[2025-08-22 06:08:14] Decode batch. #running-req: 1, #token: 291, token usage: 0.01, accept len: 1.68, cuda graph: True, gen throughput (token/s): 171.76, #queue-req: 0, 


[2025-08-22 06:08:15] Decode batch. #running-req: 1, #token: 357, token usage: 0.02, accept len: 1.65, cuda graph: True, gen throughput (token/s): 169.46, #queue-req: 0, 


[2025-08-22 06:08:15] Decode batch. #running-req: 1, #token: 425, token usage: 0.02, accept len: 1.70, cuda graph: True, gen throughput (token/s): 174.89, #queue-req: 0, 


[2025-08-22 06:08:15] Decode batch. #running-req: 1, #token: 490, token usage: 0.02, accept len: 1.62, cuda graph: True, gen throughput (token/s): 167.40, #queue-req: 0, 


[2025-08-22 06:08:16] Decode batch. #running-req: 1, #token: 563, token usage: 0.03, accept len: 1.82, cuda graph: True, gen throughput (token/s): 188.13, #queue-req: 0, 


[2025-08-22 06:08:16] Decode batch. #running-req: 1, #token: 637, token usage: 0.03, accept len: 1.85, cuda graph: True, gen throughput (token/s): 190.48, #queue-req: 0, 
[2025-08-22 06:08:16] INFO:     127.0.0.1:41514 - "POST /v1/chat/completions HTTP/1.1" 200 OK


In [16]:
terminate_process(server_process)

## References

EAGLE process is as follows:

- Within EAGLE the draft model predicts the next feature vector, i.e. the last hidden state of the original LLM, using the feature sequence $(f_1, ..., f_k)$ and the token sequence $(t_2, ..., t_{k+1})$. 
- The next token is then sampled from $p_{k+2}=\text{LMHead}(f_{k+1})$. Afterwards, the two sequences are extended in a tree style—branching out multiple potential continuations, with the branching factor per step controlled by the `speculative_eagle_topk` parameter—to ensure a more coherent connection of context, and are given as input again.
- EAGLE-2 additionally uses the draft model to evaluate how probable certain branches in the draft tree are, dynamically stopping the expansion of unlikely branches. After the expansion phase, reranking is employed to select only the top `speculative_num_draft_tokens` final nodes as draft tokens.
- EAGLE-3 removes the feature prediction objective, incorporates low and mid-layer features, and is trained in an on-policy manner.

This enhances drafting accuracy by operating on the features instead of tokens for more regular inputs and passing the tokens from the next timestep additionally to minimize randomness effects from sampling. Furthermore the dynamic adjustment of the draft tree and selection of reranked final nodes increases acceptance rate of draft tokens further. For more details see [EAGLE-2](https://arxiv.org/abs/2406.16858) and [EAGLE-3](https://arxiv.org/abs/2503.01840) paper.


For guidance how to train your own EAGLE model please see the [EAGLE repo](https://github.com/SafeAILab/EAGLE/tree/main?tab=readme-ov-file#train).